In [1]:
import kdsource as kds
import kdsource.histograms as kdh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openmc
import os
import glob


In [2]:
# Init trackfile
N_original = int(5e6)
source_path = "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/4-7-25/sim_0/trackfile_0.h5"

# Load binning data
columns_order = ["ln(E0/E)", "x", "y", "mu", "phi"]
micro_bins = [200] * 5
macro_bins = [10, 8, 8, 7]
binning_type = "equal_bins"
user_defined_edges = [
    [2.995732273553991 + 1e-9],
    # None,
    [-1.5, 1.5],
    [-1.5, 1.5],
    [1 - 1e-9],
    None,
]

# Load z0 and particle type
z0 = 30
particle_type = "neutron"

trackfile = kdh.SurfaceTrackProcessor(source_file=source_path, N_original=N_original)
trackfile.configure_binning(
    columns=columns_order,
    micro_bins=micro_bins,
    macro_bins=macro_bins,
    binning_type=binning_type,
    user_defined_macro_edges=user_defined_edges,
)
trackfile.load_info_for_onthefly(z0=z0, particle_type=particle_type)
trackfile.save_to_xml_on_the_fly()

Number of particles in file /home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/4-7-25/sim_0/trackfile_0.h5: 2677552
100% n, 0% g, 0% e-, 0% e+, 0% p, 


In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------------
# Configuración de secciones eficaces
# ---------------------------------------------------------------------------------------------------------------------------------------
openmc.config["cross_sections"] = (
    "/home/lucas/Documents/Proyecto_Integrador/endfb-viii.0-hdf5/cross_sections.xml"
)

# ----------------------------------------------------------------------------------------------------------------------------------------
# Procesamiento de la fuente
# ----------------------------------------------------------------------------------------------------------------------------------------
source = openmc.HistogramSource(path = "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/14-9-25/source.xml")

# ---------------------------------------------------------------------------------------------------------------------------------------
# Procesamiento de la geometría
# ---------------------------------------------------------------------------------------------------------------------------------------
# Extraer parámetros geométricos
vacio = True
L_x, L_y, L_z = 15,15,100
if vacio:
    L_x_vacio, L_y_vacio = 3,3

# Definir material: agua
mat_agua = openmc.Material(name="agua")
mat_agua.add_nuclide("H1", 2.0, "ao")
mat_agua.add_nuclide("O16", 1.0, "ao")
mat_agua.add_s_alpha_beta("c_H_in_H2O")
mat_agua.set_density("g/cm3", 1.0)
mats = openmc.Materials([mat_agua])
mats.export_to_xml()

# Definir superficies externas
surfaces = {
    "x_min": openmc.XPlane(x0=-L_x / 2, boundary_type="vacuum"),
    "x_max": openmc.XPlane(x0=L_x / 2, boundary_type="vacuum"),
    "y_min": openmc.YPlane(y0=-L_y / 2, boundary_type="vacuum"),
    "y_max": openmc.YPlane(y0=L_y / 2, boundary_type="vacuum"),
    "z_min": openmc.ZPlane(z0=z0, boundary_type="vacuum"),
    "z_max": openmc.ZPlane(z0=L_z, boundary_type="vacuum"),
}

# Se agrega la superficie de registro para generar el track file
surfaces.update(
    {
        "z_track": openmc.ZPlane(
            z0=60, boundary_type="transmission", surface_id=70
        )
    }
)

# Si hay vacío, definir superficies internas
if vacio:
    surfaces.update(
        {
            "x_min_vacio": openmc.XPlane(
                x0=-L_x_vacio / 2, boundary_type="transmission"
            ),
            "x_max_vacio": openmc.XPlane(
                x0=L_x_vacio / 2, boundary_type="transmission"
            ),
            "y_min_vacio": openmc.YPlane(
                y0=-L_y_vacio / 2, boundary_type="transmission"
            ),
            "y_max_vacio": openmc.YPlane(
                y0=L_y_vacio / 2, boundary_type="transmission"
            ),
        }
    )

# Para fuente tipo FileSource se traduce la superficie inferior para posicionar z0.
# Sino se hace entonces las particulas aparecer fuera de la geometria.
surfaces["z_min"].translate(vector=(0, 0, -1e-6), inplace=True)

# Definir regiones
region_externa = (
    +surfaces["x_min"]
    & -surfaces["x_max"]
    & +surfaces["y_min"]
    & -surfaces["y_max"]
    & +surfaces["z_min"]
    & -surfaces["z_max"]
)

if vacio:
    region_vacio = (
        +surfaces["x_min_vacio"]
        & -surfaces["x_max_vacio"]
        & +surfaces["y_min_vacio"]
        & -surfaces["y_max_vacio"]
        & +surfaces["z_min"]
        & -surfaces["z_max"]
    )

# Crear universo y definir celdas según configuración de fuente y vacío
universe = openmc.Universe()

universe.add_cell(
    openmc.Cell(
        region=region_externa & ~region_vacio & -surfaces["z_track"],
        fill=mat_agua,
        name="agua1",
    )
)
universe.add_cell(
    openmc.Cell(
        region=region_externa & ~region_vacio & +surfaces["z_track"],
        fill=mat_agua,
        name="agua2",
    )
)
universe.add_cell(
    openmc.Cell(
        region=region_vacio & -surfaces["z_track"],
        fill=None,
        name="vacio1",
    )
)
universe.add_cell(
    openmc.Cell(
        region=region_vacio & +surfaces["z_track"],
        fill=None,
        name="vacio2",
    )
)

geom = openmc.Geometry(universe)
geom.export_to_xml()

# ---------------------------------------------------------------------------------------------------------------------------------------
# Configuración de settings
# ---------------------------------------------------------------------------------------------------------------------------------------
settings = openmc.Settings()
settings.surf_source_write = {"surface_ids": [70], "max_particles": 20000000}
settings.run_mode = "fixed source"
settings.batches = 100
settings.particles = int(1e3)
settings.source = source

# Se definen las ventanas de peso
# Define weight window spatial mesh
ww_mesh = openmc.RegularMesh()
ww_mesh.dimension = (10, 10, 10)
ww_mesh.lower_left = (-L_x / 2, -L_y / 2, 30)
ww_mesh.upper_right = (L_x / 2, L_y / 2, L_z)

# Create weight window object and adjust parameters
wwg = openmc.WeightWindowGenerator(
    method="magic",
    mesh=ww_mesh,
    max_realizations=settings.batches,
    energy_bounds=[0, 1, 1e3, 1e7],
    update_interval=2,
)

# Add generator to Settings instance
settings.weight_window_generators = wwg
settings.export_to_xml()

# ---------------------------------------------------------------------------------------------------------------------------------------
# Configuración de tallies
# ---------------------------------------------------------------------------------------------------------------------------------------
# Tally: malla para flujo total
mesh = openmc.RectilinearMesh()
mesh.x_grid = np.linspace(-L_x / 2, L_x / 2, 2)
mesh.y_grid = np.linspace(-L_y / 2, L_y / 2, 2)
mesh.z_grid = np.linspace(30, L_z, 51)
mesh_filter = openmc.MeshFilter(mesh)
tally_flux_total = openmc.Tally(name="flux_total")
tally_flux_total.filters = [mesh_filter]
tally_flux_total.scores = ["flux"]

# Tally: malla para flujo en vacio
if vacio:
    mesh_vacio = openmc.RectilinearMesh()
    mesh_vacio.x_grid = np.linspace(-L_x_vacio / 2, L_x_vacio / 2, 2)
    mesh_vacio.y_grid = np.linspace(-L_y_vacio / 2, L_y_vacio / 2, 2)
    mesh_vacio.z_grid = np.linspace(30, L_z, 51)
    mesh_filter_vacio = openmc.MeshFilter(mesh_vacio)
    tally_flux_vacio = openmc.Tally(name="flux_vacio")
    tally_flux_vacio.filters = [mesh_filter_vacio]
    tally_flux_vacio.scores = ["flux"]

tallies = openmc.Tallies(
    [tally_flux_total, tally_flux_vacio] if vacio else [tally_flux_total]
)

# Tally: superficie para espectro en 1m
def make_spectrum_tally(L_x, L_y, L_z, name):
    mesh_total = openmc.RectilinearMesh()
    mesh_total.x_grid = np.linspace(-L_x / 2, L_x / 2, 2)
    mesh_total.y_grid = np.linspace(-L_y / 2, L_y / 2, 2)
    mesh_total.z_grid = np.linspace(L_z - 1, L_z, 2)

    tally_surface = openmc.Tally(name=name)
    tally_surface.filters = [
        openmc.MeshFilter(mesh_total),
        openmc.EnergyFilter(np.logspace(-3, 7, 50)),
    ]
    tally_surface.scores = ["flux"]
    return tally_surface

for Z in [45, 60]:
    if 30 <= Z <= L_z:
        tallies.append(
            make_spectrum_tally(L_x, L_y, Z, f"espectro_total_{Z}cm")
        )
        if vacio:
            tallies.append(
                make_spectrum_tally(
                    L_x_vacio, L_y_vacio, Z, f"espectro_vacio_{Z}cm"
                )
            )

tallies.export_to_xml()

# ---------------------------------------------------------------------------------------------------------------------------------------
# Limpieza de archivos previos y ejecución de la simulación
# ---------------------------------------------------------------------------------------------------------------------------------------
for file in glob.glob("statepoint.*.h5"):
    os.remove(file)
if os.path.exists("summary.h5"):
    os.remove("summary.h5")

openmc.run()

/home/lucas/anaconda3/envs/proyecto_integrador/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=4.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: OpenMC aborted unexpectedly.